In [1]:
import string 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
import re
import numpy as np
import emoji


def remove_emoji(string):
    return emoji.get_emoji_regexp().sub(u'', string)

def clean_text(text):
    
    text = str(text).lower() #Make text lowercase
    text = re.sub('\[.*?\]', '', text) #remove text in square brackets
    text = re.sub('https?://\S+|www\.\S+', '', text) #,remove links
    text = re.sub('<.*?>+', '', text) #remove punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) #remove punctuation
    text = re.sub('\n', '', text) #containing numbers
    text = re.sub('\w*\d\w*', '', text) #remove numbers
    text = ''.join([i for i in text if not i.isdigit()]) #remove numbers
    
    return text
    
def preprocessing(text, list_format = True):
    text = clean_text(text)
    text = remove_emoji(text)
    if list_format:
        text = text.split(" ")
    return text

In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd


ModuleNotFoundError: No module named 'amzwbscr_selenium'

In [26]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd


# using the product ID to create url
def scrape_amz(product_id):

    #start webdriver
    driver = webdriver.Chrome("/Users/jannick/Downloads/chromedriver")

    #defining the url for the first page
    page_no = 1
    url = f"https://www.amazon.com/en/product-reviews/{product_id}/ref=cm_cr_dp_d_paging_btm_next_{page_no}?ie=UTF8&reviewerType=all_reviews&pageNumber={page_no}"

    #create the lists for the data
    reviews = []
    review_metas = []

    pages = 5

    #counting no of pages, can be used for pages
    #driver.get(url)
    #soup = BeautifulSoup(driver.page_source, "html.parser")
    #review_count = soup.find("div", class_= "a-row a-spacing-base a-size-base").text
    #review_no = review_count.replace("\n", "").replace(" ","").replace(",","")
    #max_len_rno = min([13, len(review_no)])
    #review_no = review_no[:-max_len_rno].split("|")[1]
    #review_no = float(review_no)
    #pages_counted = round(review_no/10) -1
    #print(f"total pages would be {pages_counted} but we only look at {pages}")


    #go to every page
    for i in range(1, pages+1):

        #get the soup of each page
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        review_parent = soup.find_all("div", class_ = "a-row a-spacing-small review-data")

        #get each review text
        for i in review_parent:
            #review = review_parent.find("span", class_ = "cr-original-review-content")
            reviews.append(i.text.replace("\n",""))

        #get each review meta
        review_meta = soup.find_all("span",
                                   {"data-hook":True},
                                   class_="a-size-base a-color-secondary review-date")
        for i in review_meta:
            helper = i.text[12:]
            review_metas.append(helper.split(" on "))

        print(f"page {page_no} of {pages} scraped")

        #update url for next page
        page_no += 1
        url = f"https://www.amazon.com/product-reviews/{product_id}/ref=cm_cr_dp_d_paging_btm_next_{page_no}?ie=UTF8&reviewerType=all_reviews&pageNumber={page_no}"

    
    
    
    #create df
    metas_df = pd.DataFrame(review_metas)

    
    metas_df[1] = pd.to_datetime(metas_df[1])
    
    texts_df = pd.DataFrame(reviews)
    df = pd.DataFrame()
    df["country"] = metas_df[0]
    df["date"] = metas_df[1]
    df["text"] = texts_df

    return df


In [27]:
df = scrape_amz("1400079985")

/var/folders/s4/671kyfms3wnc45z745cx9dc00000gn/T/ipykernel_35988/1619940836.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/Users/jannick/Downloads/chromedriver")


page 1 of 5 scraped
page 2 of 5 scraped
page 3 of 5 scraped
page 4 of 5 scraped
page 5 of 5 scraped


In [21]:
df

,date,text
0,2018-06-18,It is unlikely that anyone who has the enduran...
1,2019-08-06,This is a knock off translation with no inform...
2,2017-02-23,"I have, at various times, tried to read four d..."
3,2020-07-01,"I bought the Kindle edition, but this applies ..."
4,2020-07-31,I wanted the Richard Pevere edition and though...
5,2019-10-22,"""Look Inside"" shows pages from Richard Pevear ..."
6,2016-08-10,This translation of War and Peace is so bad yo...
7,2022-03-20,"I read Tolstoy's ""War and Peace"" in book form ..."
8,2021-05-15,The actual book is nicely bound with thin soft...
9,2020-07-08,Tolstoy is the best. That is all you need to ...


In [8]:
df2 = pd.DataFrame()
df2["text"] = df["text"].apply(lambda x: preprocessing(x, list_format=False))
df2["date"] = df["date"]

/var/folders/s4/671kyfms3wnc45z745cx9dc00000gn/T/ipykernel_35988/1015505189.py:11: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  return emoji.get_emoji_regexp().sub(u'', string)


In [9]:
sentiments = [2,2,0,1,2,0,2,2,2,1,0,1,0,2,1,0,2,0,2,0,0,0,0,2,2,1,2,0,0,2,1,2,1,2,2,0,2,0,1,2,1,2,1,1,1,1,0,2,0,2]
sen_df = pd.DataFrame(sentiments)
df2["sentiment"] = sen_df[0]
df2

,text,date,sentiment
0,i was a little sceptical buying this phone bec...,2019-04-27,2
1,this seller is amazing was a bit skeptical at ...,2019-06-12,2
2,just was told by my carrier att that this phon...,2019-08-20,0
3,i purchased this phone for my daughter and it ...,2019-05-14,1
4,i don’t know why everyone’s giving this seller...,2019-05-21,2
5,the media could not be loa...,2019-06-10,0
6,received my iphone a few days ago and everythi...,2019-05-21,2
7,i absolutly loved this iphone xr it was wonderful,2019-04-03,2
8,i’ll admit after reading through the reviews i...,2019-08-15,2
9,update i still have this phone still works am...,2019-08-14,1


In [10]:
df2.to_csv("example_amz_rev.csv")

In [ ]:
df2.texts

In [28]:
df2

,text,date,sentiment
0,was a lttle sceptcal buyng ths phone because ...,2019-04-27,2
1,ths seller s amazng was a bt skeptcal at frst ...,2019-06-12,2
2,just was told by my carrer att that ths phone ...,2019-08-20,0
3,purchased ths phone for my daughter and t cam...,2019-05-14,1
4,don’t know why everyone’s gvng ths seller a b...,2019-05-21,2
5,the meda could not be load...,2019-06-10,0
6,receved my phone a few days ago and everythng ...,2019-05-21,2
7,absolutly loved ths phone xr t was wonderful,2019-04-03,2
8,’ll admt after readng through the revews was ...,2019-08-15,2
9,update stll have ths phone stll works amazng...,2019-08-14,1


In [ ]:
input_list = df2.values.tolist()
input_list


In [ ]:
google univeral encoder (512 dim)
simple knn clustering
t-sne

Word Cloud 
Google translater API 
